In [1]:
%load_ext autoreload
%autoreload 2


HEALTHY_DATA_PATH = "healthy_data.hdf5"
STIM_DATA_PATH = "stim_data.hdf5"
BEN_MODEL_PATH = "ben.pth"
BATCH_SIZE = 64

import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

import generate_stim_train_data
import stim_model


In [2]:
generate_stim_train_data.generate(HEALTHY_DATA_PATH, lesion=False, stim_max_power=0.0)

Generating example 0
Generating example 100
Generating example 200
Generating example 300
Generating example 400
Generating example 500
Generating example 600
Generating example 700
Generating example 800
Generating example 900
Generating example 1000
Generating example 1100
Generating example 1200
Generating example 1300
Generating example 1400
Generating example 1500
Generating example 1600
Generating example 1700
Generating example 1800
Generating example 1900


In [6]:
healthy_dataset = stim_model.StimDataset(HEALHTY_DATA_PATH)

example_len = dataset[0][0].shape[0]

assert (dataset[0][1].shape[1] % 3) == 0
module_feature_dim = dataset[0][1].shape[1] // 3
brain_feature_dim = module_feature_dim * 3
stim_dim = dataset[0][0].shape[1] - (3 * module_feature_dim)

In [ ]:
cpn = nn.Linear(brain_feature_dim, stim_dim)
ben = stim_model.StimModel(brain_feature_dim + stim_dim, brain_feature_dim)
brain = mRNN.MichaelsRNN(
        init_data_path=generate_stim_train_data.network_path,
        num_stim_channels=generate_stim_train_data.NUM_STIM_ELECTRODES
        )

ben.load_state_dict(torch.load(BEN_MODEL_PATH))
ben.eval()

# Lesion 75% of F5
lesion_mask = torch.ones((brain.num_neurons,))
lesion_mask[brain.num_neurons_per_module : brain.num_neurons_per_module] = 0.0
brain.set_lesion(lesion_mask)

    
optimizer = Adam(model.parameters(), lr=0.008)

In [42]:
torch.autograd.set_detect_anomaly(True)

losses = []
min_loss = None
for _ in range(2000):
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    for i_batch, sampled_batch in enumerate(loader):
        cpn.reset()
        ben.reset()
        optimizer.zero_grad()
        
        din, dout = sampled_batch
        batch_size = din.shape[0]
        
        brain_din = din[:, :brain_feature_dim]
                 
        ben_preds = torch.empty((batch_size, example_len, brain_feature_dim))
        for tidx in range(example_len):
            cur_in = din[:, tidx, :]
            pred1 = cpn(cur_in)
            pred1_padded = torch.zeros((pred1.shape[0], in_dim))
            pred1_padded[:, :10] = pred1[:,:]
            pred = sm(pred1_padded)
                        
            preds[:, tidx, :] = pred.reshape((1,) + pred.shape)[:, :]
            
        loss = torch.nn.MSELoss()(ben_preds, dout)
        losses.append(loss.item())
        print(i_batch, loss)
        
        if min_loss is None:
            min_loss = loss.item()
        elif min_loss > loss.item():
            torch.save(cpn.state_dict(), "cpn_model.pth")
            min_loss = loss.item()
                
        loss.backward()
        optimizer.step()

None
0 tensor(0.1204, grad_fn=<MseLossBackward>)
tensor([[-9.7216e-04, -6.8758e-04, -3.3728e-04,  ...,  2.9421e-04,
          3.5721e-04, -6.4331e-04],
        [-2.3303e-03, -1.6674e-03, -8.3013e-04,  ...,  5.9735e-04,
          1.7399e-04, -1.2924e-03],
        [-8.6358e-04, -6.3953e-04, -3.0043e-04,  ...,  2.2660e-04,
          6.1363e-04, -1.2284e-04],
        ...,
        [-6.8300e-04, -4.7923e-04, -2.7855e-04,  ..., -2.6373e-04,
         -4.0816e-05, -9.0174e-04],
        [-3.1498e-04, -2.2471e-04, -7.1246e-05,  ..., -9.9359e-06,
         -1.0447e-04, -6.9460e-06],
        [ 1.6365e-04,  1.4940e-04,  1.2058e-04,  ...,  3.0128e-05,
          1.4510e-04, -2.3275e-04]])
